# DTW Algorithm, Retrieval and Evaluation (Recall, precision, AP)

In [ ]:
import pandas as pd
import numpy as np
import os
import pickle
from tslearn.metrics import dtw
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
import matplotlib.pyplot as plt

## Load Data

### Feature vectors

In [ ]:
from FeatureVectors import load_images
from FeatureVectors import getFeaturesMatrix
results, numbers = load_images()
allfeatures = getFeaturesMatrix(numbers, results.keys())

In [ ]:
# replace every none value by 0 for DTW calculation
for key in allfeatures.keys():
    length = len(allfeatures[key])
    for line in range(0,length):
        allfeatures[key][line]=[0.0 if v is None else v for v in allfeatures[key][line]]

In [ ]:
allfeatures['270-01-01']

### Train and validation set

In [ ]:
# task folder locations
dataset = os.path.join('..', 'dataset')
task_fold = os.path.join(dataset, 'task')
# training list
train_f = open(os.path.join(task_fold, 'train.txt'), "r")
train_list = []
for l in train_f:
    train_list.append(l[:3])
# validation list
val_f = open(os.path.join(task_fold, 'valid.txt'), "r")
val_list = []
for l in val_f:
    val_list.append(l[:3])
# create training and validation dataset
train_dataset = {}
val_dataset = {}
for key in allfeatures.keys():
    if key[:3] in train_list:
        train_dataset[key] = allfeatures[key]
    elif key[:3] in val_list:
        val_dataset[key] = allfeatures[key]
    else:
        print(f'{key} is not included in training or validation dataset')

print('train_dataset and val_dataset created')

### Transcription

In [ ]:
# ground-truth folder location
groundtruth = os.path.join(dataset, 'ground-truth')
transcription_f = open(os.path.join(groundtruth, 'transcription.txt'), "r")
transcription = {}
for l in transcription_f:
    key_word = l.split()
    transcription[key_word[0]] = key_word[1]
print('Transcriptions loaded.')

In [ ]:
transcription['270-03-01']

In [ ]:
#load the words that appear both in training and validation set
keywords_f = open(os.path.join(task_fold, 'keywords.txt'), "r")
keywords_list = []
for l in keywords_f:
    keywords_list.append(l[:len(l)-1])
print('Keywords loaded.')

## DTW Algorithm

References :
* [Install tslearn](https://tslearn.readthedocs.io/en/latest/installation.html)
* [tslearn.metrics](https://tslearn.readthedocs.io/en/latest/gen_modules/tslearn.metrics.html)


In [ ]:
cost_results = {}
#check if we already calculated the results, if not do it
file = os.path.join('cost_results', 'results.pkl')
if not os.path.isfile(file):
    #warning: the following two for loops takes a lot of time to compute
    for val_vector in val_dataset:
        # !!! uncomment the following line and add increment such that it works if you want to compute only meaningful results !!!
        # if transcription[val_vector] in keywords_list:
        cost_results[val_vector] = {}
        for train_vector in train_dataset:
            cost = dtw(val_dataset[val_vector],
                       train_dataset[train_vector],
                       global_constraint="sakoe_chiba",
                       sakoe_chiba_radius=3)
            cost_results[val_vector][train_vector] = cost
    #save the cost results
    with open('cost_results/results.pkl', 'wb') as f:
        pickle.dump(cost_results, f, pickle.HIGHEST_PROTOCOL)
    print('DTW results saved')
else:
    #if the results already exists, load them
    with open(file, 'rb') as f:
        cost_results = pickle.load(f)
    print('DTW results loaded')
    

In [ ]:
cost_results['300-02-01']

## Evaluation

### The idea :
Define thresholds k for considering a certain cost as a limit
to choose results below the limit as positive (similar to the test word) 
and after the limit as negative (not similar to the test word)

Then for each k:

a)  check from top to k if it is a true or false positive 
    by comparing the transcription of the test word to the one of the train word
    
    => compute true positive and false positive
    
b)  check from k to the end if it is a true or false negative
    by comparing the transcription of the test word to the one of the train word
    
    => compute true negative and false negative
    
c)  compute: 

    Precision = True Positives / (True Positives + False Positives)
    
    Recall = True Positives / (True Positives + False Negatives)
    
    => add [precision,recall] to a global list with the values obtained for each k
    
    => then compute the mean for precision and recall

### The implementation :

#### Sort the results with most similar one on top

In [ ]:
def sort_dict(dictionary):
    result = {k: v for k, v in sorted(dictionary.items(), key=lambda item: item[1])}
    return result

In [ ]:
# sort the cost_results
for word in cost_results.keys():
    cost_results[word] = sort_dict(cost_results[word])

In [ ]:
cost_results['300-02-01']

#### Binarize the results into either positive (1) or negative (0) with ground truth and keep words that appear at least once into train and validation dataset

In [ ]:
binarized_results = {}
# final_results is the dictionary that contains only meaningful results
final_results = {}
for val_word in cost_results.keys():
    binarized_list = []
    val_transcription = transcription[val_word]
    #keep only the words that appear both in validation and training set
    if val_transcription in keywords_list:
        final_results[val_word] = cost_results[val_word]
        for train_word in cost_results[val_word].keys():
            train_transcription = transcription[train_word]
            if val_transcription == train_transcription:
                binarized_list.append(1)
            else:
                binarized_list.append(0)
        binarized_results[val_word] = binarized_list
print('Results are now filtered and binarised into positive or negative retrieval.')

#### Compute precision and recall for every word and for every threshold k

In [ ]:
# This takes some time to compute, don't worry (1-2 min max)
precision_recall_data = {}
for word in final_results.keys():
    lr_precision = []
    lr_recall = []
    for k in range(0, len(final_results[word])):
        tp = 0
        fp = 0
        tn = 0
        fn = 0
        for r in range(0,k):
            if binarized_results[word][r] == 1:
                tp = tp + 1
            else:
                fp = fp + 1
        for r in range(k, len(final_results[word])):
            if binarized_results[word][r] == 1:
                fn = fn + 1
            else:
                tn = tn + 1
        if (tp+fp) == 0:
            precision = 0
        else:
            precision = tp / (tp + fp)
        if (tp+fn) == 0:
            recall = 0
        else:
            recall = tp / (tp + fn)
        lr_precision.append(precision)
        lr_recall.append(recall)
    precision_recall_data[word] = [lr_precision, lr_recall]

#### Plot the precision-recall curve for every word

In [ ]:
for word in final_results.keys():
    lr_precision, lr_recall = precision_recall_data[word][0], precision_recall_data[word][1]
    lr_auc = auc(lr_recall, lr_precision)
    print(f'Word : {word} ({transcription[word]})')
    print('Logistic: AP=%.3f' % lr_auc)

    plt.plot(lr_recall, lr_precision, marker='.', label='Logistic')
    # axis labels
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    # show the legend
    plt.legend()
    # show the plot
    plt.show()

#### Plot the final precision-recall curve (average of all the words)

In [ ]:
# initialize mean precision and mean recall
lr_mean_precision = []
lr_mean_recall = []
for i in range(0, len(final_results[word])):
    lr_mean_precision.append(0)
    lr_mean_recall.append(0)
# compute the mean precision and recall   
for word in final_results.keys():
    lr_precision, lr_recall = precision_recall_data[word][0], precision_recall_data[word][1]
    lr_mean_precision = [x + y for x, y in zip(lr_mean_precision, lr_precision)]
    lr_mean_recall = [x + y for x, y in zip(lr_mean_recall, lr_recall)]
    
lr_mean_precision= [idx/len(final_results) for idx in lr_mean_precision]
lr_mean_recall= [idx/len(final_results) for idx in lr_mean_recall]

# plot the results
lr_mean_auc = auc(lr_mean_recall, lr_mean_precision)
print('Logistic: AP=%.3f' % lr_mean_auc)
plt.plot(lr_mean_recall, lr_mean_precision, marker='.', label='Logistic')
# axis labels
plt.xlabel('Recall')
plt.ylabel('Precision')
# show the legend
plt.legend()
# show the plot
plt.show()